In [1]:
from hello.hello3 import HelloApp, NotLoggedInError, BadError
import time, datetime
import requests

In [5]:
def wait1(s):
    dend = (datetime.datetime.now() + datetime.timedelta(seconds=s)).strftime("%m/%d/%Y %I:%M:%S %p")
    print("Sleeping %d seconds, ending at %s" %(s,dend))
    start = time.time()
    end = start + s
    try:
        while time.time() < end:
            time.sleep(5)
    except KeyboardInterrupt:
        pass
    
def call(f, *args):
    s = "(" + ", ".join(map(repr, args)) + ")"
    print("Calling %s%s" % (f.__name__, s))
    while True:
        try:
            rv = f(*args)
        except NotLoggedInError:
            h.login()
        except requests.exceptions.ConnectTimeout as e:
            print(str(e))
            print("Make sure you're on the right network!")
        except (BadError, IOError) as e:
            print(str(e))
        else:
            return rv
        
def startbatch(name):
    if call(h.batchrunning):
        call(h.endbatch)
    return call(h.startbatch, name)

def setdo(mode, n2_or_sp, o2=None):
    call(h.setdo, mode, n2_or_sp,o2)
    
def doman(n2, o2):
    setdo(1, n2, o2)
    
def doauto(sp):
    setdo(0, sp)
    
def dooff():
    setdo(2, 0, 0)
        
def endbatch():
    call(h.endbatch)

def wait2(s):
    pass
wait = wait1

In [ ]:
h = HelloApp('71.189.82.196:85')
wait = wait1
dooff()
wait(3600*10)
startbatch("dopid 1.1")
doauto(150)
wait(3600*20)
endbatch()
startbatch("dopid 1.2")
doauto(100)
wait(3600*20)
endbatch()
startbatch("dopid 1.3")
doauto(50)
wait(3600*20)
endbatch()

Calling setdo(2, 0, 0)
Sleeping 36000 seconds, ending at 05/06/2017 04:02:38 AM
